In [1]:
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
import numpy as np
from pandas import DataFrame
from scipy.stats.distributions import  t

xl = pd.ExcelFile('demo_data.xls')
df = pd.read_excel(xl, 0, header=0)

results = smf.ols('y ~ X', data=df).fit()
# Các biến trên công thức trên nó phải là tên cột của df

In [2]:
X = df['X'].to_numpy() # Chuyển sang Numpy Array
X = np.stack((np.ones(X.shape[0]), X), axis=0).T
m = X.shape[0] # Số mẫu
n = X.shape[1] - 1 # Số biến (slope)
y = df['y'].to_numpy() # Chuyển sang Numpy Array

In [3]:
X_bar = np.mean(X, axis=0) # Trung bình cộng
y_bar = np.mean(y) # Trung bình cộng toàn bộ phần tử

XX = X.T @ X # @ là phép nhân ma trận
XX_inv = np.linalg.inv(XX) # Ma trận nghịch đảo
theta = XX_inv @ X.T @ y # Công thức số (7) trang 9
y_hat = X @ theta # Công thức số (5) trang 8

In [4]:
RSS = np.sum((y - y_hat)**2) # Công thức trang 12
# residual sum of squares

ESS = np.sum((y_hat - y_bar)**2) # Công thức trang 12
# explained sum of squares

TSS = np.sum((y - y_bar)**2)
# total sum of square

In [5]:
R2 = 1 - RSS / TSS # R-squared
R2_adj = (R2 * (m - 1) - n) / (m - n - 1) # Adj. R-squared

S2_n = RSS / (m - n - 1) #  is an unbiased estimator of the variance of the error
S2_theta = S2_n * np.diag(XX_inv) # ma trận hiệp phương sai
S_theta = np.sqrt(S2_theta) # standard error

F_value = (ESS / n) / (RSS / (m - n - 1)) #
# F_value càng lớn càng tốt
# ESS: giải thích được (explain sum of squares)
# RSS: không giải thích được
# tỷ lệ ESS/RSS càng lớn lớn thì mô hình thiết kế có ý nghĩa
# F_value => ESS = 0, chúng ta không giải thích được gì
# F_value càng lớn, chúng ta được nhiều


F_p_value = (1 - stats.f.cdf(F_value, n, m - n - 1)) #

In [6]:
F_p_value = (1 - stats.f.cdf(F_value, n, m - n - 1))
# cumulative distribution function
# input: giá trị x = F_value
# output: cumulative probability
# thư viện stats có phân phối f
# phân phối f có hàm cumulative distribution function (cdf)
print(F_p_value)

1.1102230246251565e-16


In [7]:
F_value, F_p_value

(np.float64(268.5867127965644), np.float64(1.1102230246251565e-16))

In [8]:
t_values = theta / S_theta
t_p_values = (1 - stats.t.cdf(abs(t_values), m - n - 1)) * 2

In [9]:
t_p_values

array([0., 0.])

In [10]:
alpha = 0.05
sT = t.ppf(1.0 - alpha/2.0, m - n - 1)
theta_CI = sT * S_theta

In [11]:
print('R-squared', R2)
print('Adj. R-squared', R2_adj)

R-squared 0.7326689795917815
Adj. R-squared 0.7299411120365955


In [12]:
df_temp = pd.DataFrame(data = {'coef': theta,
                               'std err': S_theta,
                               't': t_values,
                               'P>|t|': t_p_values,
                               '[0.025': theta - theta_CI,
                               '0.975]': theta + theta_CI},
                       index = ['Intercept', 'X'])

display(df_temp.round(4))

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,4.1648,0.2031,20.5034,0.0,3.7617,4.5680
X,2.9328,0.1790,16.3886,0.0,2.5776,3.2879


In [13]:
F_value, F_p_value

(np.float64(268.5867127965644), np.float64(1.1102230246251565e-16))

In [14]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.733
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     268.6
Date:                Thu, 26 Sep 2024   Prob (F-statistic):           7.88e-30
Time:                        13:11:00   Log-Likelihood:                -147.81
No. Observations:                 100   AIC:                             299.6
Df Residuals:                      98   BIC:                             304.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.1648      0.203     20.503      0.0